# freud.locality.Filter

In this notebook, we introduce the neighborlist filter concept in freud, discuss the filtering methods implemented in freud, and demonstrate how to use them with good efficiency.

## What is a NeighborList Filter?

A neighborlist filter is a class which removes some bonds from a pre-defined neighborlist. A neighborlist filter can be thought of as a function for which an unfiltered `NeighborList` is given as the input and a filtered version of the input `NeighborList` is the output. Freud already has some methods in the `NeighborList` class like `filter` and `filter_r` which can achieve this concept, but often the full system definition is needed to decide which bonds to remove. This is often the case when typical neighbor queries which use `r_max` or `num_neighbors` label two particles as neighbors even though they are blocked by another particle in between them. Each of freud's neighborlist filters defines the concept of "blocking" or being "blocked" differently and removes bonds between particles which are blocked by another particle.

## The Solid Angle Nearest Neighbor (SANN) Method

With the SANN method, we look for enough nearby neighbors to fully occupy the $4 \pi$ solid angle distribution surrounding a particle and label all neighbors further away as blocked. Strictly speaking, we consider neighbors going radially outward and define the solid angle occupied by each neighbor as [show the formula from the paper]. The `freud.locality.FilterSANN` class implements this method, and an example usage is shown below:

In [ ]:
import freud

# make the system to operate on
N = 1000
L = 100
box, points = freud.data.make_random_system(N, L)

# create the unfiltered neighborlist
nlist = freud.locality.AABBQuery(box, points).compute(points, neighbors=dict(r_max=49, exclude_ii=True)).toNeighborList()

# make the FilterSANN and call compute
sann = freud.locality.FilterSANN()
sann.compute((box, points), neighbors=nlist)

# access the filtered neighborlist as a property at the end
sann.filtered_nlist

Optionally, instead of having to do the neighbor query explicitly in your python script, freud will do it automatically if query arguents are passed to the `neighbors` argument of `FilterSANN`'s `compute` method. The following script is equivalent to the previous script:

In [ ]:
import freud

# make the system to operate on
N = 1000
L = 100
box, points = freud.data.make_random_system(N, L)

# make the FilterSANN and call compute
sann = freud.locality.FilterSANN()
sann.compute((box, points), neighbors=dict(r_max=49, exclude_ii=True))

# get the filtered neighborlist at the end
sann.filtered_nlist

# get the unfiltered neighborlist automatically computed by freud
sann.unfiltered_nlist

In general, we recommend using `exclude_ii=True` in neighbor queries to generate the unfiltered neighborlist, due to algorithmic issues when the distance between neighbors is $0$. Custom `NeighborList`'s are also supported as inputs to the `neighbors` argument, as shown below:

In [ ]:
import freud
import numpy as np

# make the system to operate on
N = 1000
L = 100
box, points = freud.data.make_random_system(N, L)

# custom neighborlist
point_indices = np.random.randint(low=0, high=N, num=N)
query_point_indices = np.random.randint(low=0, high=N, num=N)
distances = box.wrap(np.linalg.norm(points[point_indices] - points[query_point_indices], axis=-1))
nlist = freud.locality.NeighborList.from_arrays(N, N, point_indices, query_point_indices, distances)

# make the FilterSANN and call compute
sann = freud.locality.FilterSANN()
sann.compute((box, points), neighbors=nlist)

# get the filtered neighborlist at the end
sann.filtered_nlist

If nothing is given to the `neighbors` argument, freud will automatically compute an all pairs neighborlist (excluding ii  pairs) as the unfiltered neighborlist, equivalent to what is shown below:

In [ ]:
import freud

# make the system to operate on
N = 1000
L = 100
box, points = freud.data.make_random_system(N, L)

# all pairs neighborlist
nlist = freud.locality.NeighborList.all_pairs(points, exclude_ii=True)

# make the FilterSANN and call compute
sann = freud.locality.FilterSANN()
sann.compute((box, points), neighbors=nlist)

# get the filtered neighborlist at the end
sann.filtered_nlist

## The Relative Angular Distance (RAD) Method

With the RAD method, we label a neighbor as blocked if the angle formed between it, the other neighbor, and a blocking particle is less than a given threshold.

## Incomplete Shells and Performance Considerations